In [7]:
import renom as rm
from renom.cuda import set_cuda_active
import numpy as np
from sklearn.datasets import fetch_mldata
from sklearn.preprocessing import LabelBinarizer

set_cuda_active(True)

In [8]:
cnn = rm.Sequential([
    rm.Conv2d(channel=32, filter=3, padding=1),
    rm.Relu(),
    rm.Conv2d(channel=64, filter=3, padding=1),
    rm.Relu(),
    rm.MaxPool2d(filter=2, stride=2),
    rm.Dropout(0.5),
    rm.Flatten(),
    rm.Dense(128),
    rm.Relu(),
    rm.Dense(10)
])


In [9]:
mnist = fetch_mldata('MNIST original', data_home='./')
data = mnist['data'].astype(np.float32)
data /= 256

train_num = int(0.8 * len(data))

train_data = np.expand_dims(data[:train_num].reshape(train_num, 28, 28), axis=1)

targets = mnist['target']
train_targets = targets[:train_num]
train_targets = LabelBinarizer().fit_transform(train_targets).astype(np.float32)



In [10]:
optimizer = rm.Adam()
epochs = 5
batch_size = 128

for e in range(epochs):
    loss = 0
    perm = np.random.permutation(train_num)
    
    for i in range(train_num // batch_size):
        train_batch = train_data[perm[i * batch_size:(i + 1) * batch_size]]
        train_targets_batch = train_targets[perm[i * batch_size:(i + 1) * batch_size]]
        with cnn.train():
            l = rm.softmax_cross_entropy(cnn(train_batch), train_targets_batch)
        
        l.grad().update(optimizer)
        loss += l.as_ndarray()
    print(loss/i)

[0.18043837]
[0.05359168]
[0.03749543]
[0.03067516]
[0.02220922]


In [15]:
import renom.utility.onnx
x = np.random.random((1, 1, 28, 28))
renom.utility.onnx.export_onnx("mnist", cnn, x, "mnist.onnx") 
